In [2]:
# Let's start by retrieving some Magic card images using the Scryfall API
# https://scryfall.com/docs/api
# We'll be using the Scryfall python library
# https://github.com/NandaScott/Scrython

In [103]:
import requests
from requests.exceptions import TooManyRedirects, ReadTimeout, ConnectionError
import json
import pprint
import time
import shutil
import os
import os.path
import glob
import csv
from PIL import Image
import random
import numpy as np
import errno

URL='https://api.scryfall.com'
OUTDIR='/Users/agussman/Documents/DeepLens/deeplens-magiccards/data'


In [89]:
r = requests.get('https://api.scryfall.com/cards/search?order=set&q=e:dom')
# TODO: Support Scryfall pagination: https://scryfall.com/docs/api/lists

In [90]:
results = r.json()['data']
for result in results[:1000]:
    #card = result
    #if 'all_parts' in result
    #print(result['image_uris']['png'])
    time.sleep(0.25)
    image_url = result['image_uris']['png']
    filename = "{}_{num:03d}.png".format(result['set'], num=int(result['collector_number']))
    filename = os.path.join(OUTDIR, result['set'], filename)
    print(filename)
    print("{} -> {}".format(image_url, filename))
    
    r = requests.get(image_url, stream=True)
    
    with open(filename, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=128):
            fd.write(chunk)
    
    '''
    if r.status_code == 200:
        with open(os.path.join(OUTDIR, result['set'], filename), 'wb') as out_file:
            shutil.copyfileobj(response.raw(), out_file)
    else:
        print("Nope. {}".format(response.status_code))
    '''
    
    
#pprint.pprint(results[0:2])

/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_001.png
https://img.scryfall.com/cards/png/en/dom/1.png?1524789999 -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_001.png
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_002.png
https://img.scryfall.com/cards/png/en/dom/2.png?1525862131 -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_002.png
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_003.png
https://img.scryfall.com/cards/png/en/dom/3.png?1525862133 -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_003.png
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_004.png
https://img.scryfall.com/cards/png/en/dom/4.png?1524790027 -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_004.png
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_005.png
https://img.scryfall.com/cards/png/en/dom/5.png?1528770903 -> /Users

/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_040.png
https://img.scryfall.com/cards/png/en/dom/40.png?1524790396 -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_040.png
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_041.png
https://img.scryfall.com/cards/png/en/dom/41.png?1524790406 -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_041.png
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_042.png
https://img.scryfall.com/cards/png/en/dom/42.png?1524790414 -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_042.png
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_043.png
https://img.scryfall.com/cards/png/en/dom/43.png?1524790424 -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_043.png
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_044.png
https://img.scryfall.com/cards/png/en/dom/44.png?1524790433 -> /

/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_079.png
https://img.scryfall.com/cards/png/en/dom/79.png?1524790762 -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_079.png
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_080.png
https://img.scryfall.com/cards/png/en/dom/80.png?1524790770 -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_080.png
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_081.png
https://img.scryfall.com/cards/png/en/dom/81.png?1524790780 -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_081.png
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_082.png
https://img.scryfall.com/cards/png/en/dom/82.png?1524790788 -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_082.png
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_083.png
https://img.scryfall.com/cards/png/en/dom/83.png?1524790796 -> /

/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_118.png
https://img.scryfall.com/cards/png/en/dom/118.png?1524791156 -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_118.png
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_119.png
https://img.scryfall.com/cards/png/en/dom/119.png?1525864604 -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_119.png
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_120.png
https://img.scryfall.com/cards/png/en/dom/120.png?1524791173 -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_120.png
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_121.png
https://img.scryfall.com/cards/png/en/dom/121.png?1524791187 -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_121.png
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_122.png
https://img.scryfall.com/cards/png/en/dom/122.png?1524791196

/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_156.png
https://img.scryfall.com/cards/png/en/dom/156.png?1524791556 -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_156.png
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_157.png
https://img.scryfall.com/cards/png/en/dom/157.png?1524791565 -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_157.png
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_158.png
https://img.scryfall.com/cards/png/en/dom/158.png?1524791574 -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_158.png
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_159.png
https://img.scryfall.com/cards/png/en/dom/159.png?1525862754 -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_159.png
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_160.png
https://img.scryfall.com/cards/png/en/dom/160.png?1524791593

In [73]:
# Next let's download some pictures of tables
# TODO: Don't download HTML as a JPG
table_list_url = 'http://image-net.org/api/text/imagenet.synset.geturls?wnid=n03201035'
r = requests.get(table_list_url)

file_urls = r.text.splitlines()

In [87]:
for num, table_url in enumerate(file_urls[100:]):
    print(num, table_url)
    time.sleep(0.25)
    filename = "{num:003d}_{original}".format(num=num, original=os.path.basename(table_url))
    filename = os.path.join(OUTDIR, 'tables', filename)
    print(filename)
    print("{} -> {}".format(table_url, filename))
    
    try:
        r = requests.get(table_url, stream=True, timeout=1)
        if r.status_code == 200:
            with open(filename, 'wb') as fd:
                for chunk in r.iter_content(chunk_size=128):
                    fd.write(chunk)
        else:
            print("SKIP: Problem downloading: {}".format(r.status_code))
    except TooManyRedirects:
        print("SKIP: TooManyRedirects")
    except ReadTimeout:
        print("SKIP: ReadTimeout")
    except ConnectionError:
        print("SKIP: ConnectionError")
    


0 http://farm2.static.flickr.com/1395/1474970283_e5a687c70c.jpg
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/000_1474970283_e5a687c70c.jpg
http://farm2.static.flickr.com/1395/1474970283_e5a687c70c.jpg -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/000_1474970283_e5a687c70c.jpg
1 http://peoplesfurniture.net/images_products/6228_small.jpg
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/001_6228_small.jpg
http://peoplesfurniture.net/images_products/6228_small.jpg -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/001_6228_small.jpg
SKIP: ReadTimeout
2 http://farm1.static.flickr.com/109/277350299_351613348e.jpg
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/002_277350299_351613348e.jpg
http://farm1.static.flickr.com/109/277350299_351613348e.jpg -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/002_277350299_351613348e.jpg
3 http://www.gowfb.com/images/imagecache/462-Bo

25 http://farm2.static.flickr.com/1106/1093235054_e664bec93e.jpg
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/025_1093235054_e664bec93e.jpg
http://farm2.static.flickr.com/1106/1093235054_e664bec93e.jpg -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/025_1093235054_e664bec93e.jpg
26 http://z.about.com/d/collectibles/1/5/5/_/wilmes10.jpg
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/026_wilmes10.jpg
http://z.about.com/d/collectibles/1/5/5/_/wilmes10.jpg -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/026_wilmes10.jpg
SKIP: ConnectionError
27 http://peoplesfurniture.net/images_products/4450_small.jpg
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/027_4450_small.jpg
http://peoplesfurniture.net/images_products/4450_small.jpg -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/027_4450_small.jpg
SKIP: ReadTimeout
28 http://bp1.blogger.com/_pU3onm4o7kw/RyfXslOSQ1I/AAAAA

/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/049_311957682_9375af06b8.jpg
http://static.flickr.com/101/311957682_9375af06b8.jpg -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/049_311957682_9375af06b8.jpg
50 http://farm2.static.flickr.com/1202/1107251587_8cf268f60d.jpg
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/050_1107251587_8cf268f60d.jpg
http://farm2.static.flickr.com/1202/1107251587_8cf268f60d.jpg -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/050_1107251587_8cf268f60d.jpg
51 http://www.shrivenham.com/Images/Dining.jpg
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/051_Dining.jpg
http://www.shrivenham.com/Images/Dining.jpg -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/051_Dining.jpg
52 http://www.gowfb.ca/images/imagecache/basra.jpg
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/052_basra.jpg
http://www.gowfb.ca/images/imagecache

75 http://www.staxchairs.com.au/productimg/Redang_Dining_ChairsMain1.jpg
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/075_Redang_Dining_ChairsMain1.jpg
http://www.staxchairs.com.au/productimg/Redang_Dining_ChairsMain1.jpg -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/075_Redang_Dining_ChairsMain1.jpg
SKIP: Problem downloading: 404
76 http://www.peoplesfurniture.net/images_products/5642_big.jpg
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/076_5642_big.jpg
http://www.peoplesfurniture.net/images_products/5642_big.jpg -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/076_5642_big.jpg
SKIP: ReadTimeout
77 http://www.gowfb.com/images/imagecache/5%20PC%20Round%20Dining%20Table%20Set%20-%20Walnut%20Finish.jpg
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/077_5%20PC%20Round%20Dining%20Table%20Set%20-%20Walnut%20Finish.jpg
http://www.gowfb.com/images/imagecache/5%20PC%20Round%20Dining%20T

99 http://farm1.static.flickr.com/67/217786870_d416932c49.jpg
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/099_217786870_d416932c49.jpg
http://farm1.static.flickr.com/67/217786870_d416932c49.jpg -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/099_217786870_d416932c49.jpg
100 http://www.ambfurniture.com/catalog/image.php?type=P&amp;id=18704
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/100_image.php?type=P&amp;id=18704
http://www.ambfurniture.com/catalog/image.php?type=P&amp;id=18704 -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/100_image.php?type=P&amp;id=18704
SKIP: Problem downloading: 404
101 http://farm1.static.flickr.com/76/187452261_fadd6bfdc8.jpg
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/101_187452261_fadd6bfdc8.jpg
http://farm1.static.flickr.com/76/187452261_fadd6bfdc8.jpg -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/101_187452261_fadd6bfdc8

...

/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/1541_solid_wood_dining_room_furniture.jpg
http://www.peoplesfurniture.net/scart/public/scart/images_templ/furniture_resources/solid_wood_dining_room_furniture.jpg -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/1541_solid_wood_dining_room_furniture.jpg
SKIP: ReadTimeout
1542 http://upsidedowncats.com/uploads/27744_thanksgiving_dining_room_table-20071121220305.jpg
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/1542_27744_thanksgiving_dining_room_table-20071121220305.jpg
http://upsidedowncats.com/uploads/27744_thanksgiving_dining_room_table-20071121220305.jpg -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/1542_27744_thanksgiving_dining_room_table-20071121220305.jpg
1543 http://www.dumonds.com/images/dining_room_furniture/dining_table_walnut_welch_A.jpg
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/1543_dining_table_walnut_welch_A.jpg
htt

SKIP: ConnectionError
1564 http://www.gowfb.com/images/imagecache/1258.jpg
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/1564_1258.jpg
http://www.gowfb.com/images/imagecache/1258.jpg -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/1564_1258.jpg
SKIP: Problem downloading: 404
1565 http://www.roomsbyus.com/furniture/Casual-dining-Kitchen/casual-dining/01-120581-SETA.jpg
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/1565_01-120581-SETA.jpg
http://www.roomsbyus.com/furniture/Casual-dining-Kitchen/casual-dining/01-120581-SETA.jpg -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/1565_01-120581-SETA.jpg
SKIP: Problem downloading: 400
1566 http://oxford.gumtree.com/posting_images/92/21241692__1205153688__1__1-b3e544b333e01920c885c1449e47ee9a.__big__.jpg
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/1566_21241692__1205153688__1__1-b3e544b333e01920c885c1449e47ee9a.__big__.jpg
http://oxford.g

1587 http://farm1.static.flickr.com/158/395827087_f6c8a5f8ef.jpg
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/1587_395827087_f6c8a5f8ef.jpg
http://farm1.static.flickr.com/158/395827087_f6c8a5f8ef.jpg -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/1587_395827087_f6c8a5f8ef.jpg
1588 http://farm2.static.flickr.com/1262/749409935_f9b651512e.jpg
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/1588_749409935_f9b651512e.jpg
http://farm2.static.flickr.com/1262/749409935_f9b651512e.jpg -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/1588_749409935_f9b651512e.jpg
1589 http://www.mattressmack.com/macks_weblog/images/2007/12/23/344320700.jpg
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/1589_344320700.jpg
http://www.mattressmack.com/macks_weblog/images/2007/12/23/344320700.jpg -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/1589_344320700.jpg
1590 http://www.morfurniture

1612 http://furniturefromhome.com/images/store/17900_st_pierre_leg_table_buffet_splat_chair_gd_copy/280x260/image.jpg
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/1612_image.jpg
http://furniturefromhome.com/images/store/17900_st_pierre_leg_table_buffet_splat_chair_gd_copy/280x260/image.jpg -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/1612_image.jpg
1613 http://farm3.static.flickr.com/2353/2219344458_699aeed6fe.jpg
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/1613_2219344458_699aeed6fe.jpg
http://farm3.static.flickr.com/2353/2219344458_699aeed6fe.jpg -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/1613_2219344458_699aeed6fe.jpg
1614 http://static.flickr.com/16/20245702_d5a3ee29b6.jpg
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/1614_20245702_d5a3ee29b6.jpg
http://static.flickr.com/16/20245702_d5a3ee29b6.jpg -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables

MissingSchema: Invalid URL '': No schema supplied. Perhaps you meant http://?

In [112]:
def compose_images(foreground_path, background_path):
    # Make sure the foreground path is valid and open the image
    assert os.path.exists(foreground_path), 'image path does not exist: {}'.format(foreground_path)
    assert os.path.splitext(foreground_path)[1].lower() == '.png', 'foreground must be a .png file'
    foreground = Image.open(foreground_path)
    foreground_alpha = np.array(foreground.getchannel(3))
    assert np.any(foreground_alpha == 0), 'foreground needs to have some transparency: {}'.format(foreground_path)
    
    # Make sure the background path is valid and open the image
    assert os.path.exists(background_path), 'image path does not exist: {}'.format(background_path)
    assert os.path.splitext(background_path)[1].lower() in ['.png', '.jpg', 'jpeg'], \
        'foreground must be a .png or .jpg file: {}'.format(foreground_path)
    background = Image.open(background_path)
    background = background.convert('RGBA')
    
    # Rotate the foreground
    angle_degrees = random.randint(0, 359)
    # TODO: try
    # 1/4 of the time keep it close to center
    # 1/4 of the time keep it 90 to 180
    # 1/4 of the time -180 to 180
    # 1/4 of the time anywhere
    foreground = foreground.rotate(angle_degrees, resample=Image.BICUBIC, expand=True)
    
    # Scale the foreground
    max_xy_position = (background.size[0] - foreground.size[0], background.size[1] - foreground.size[1])
    while (max_xy_position[0] < 0 or max_xy_position[1] < 0):
        print("Scaling down...")
        scale = random.random() * .5 + .5 # Pick something between .5 and 1
        new_size = (int(foreground.size[0] * scale), int(foreground.size[1] * scale))
        foreground = foreground.resize(new_size, resample=Image.BICUBIC)
        max_xy_position = (background.size[0] - foreground.size[0], background.size[1] - foreground.size[1])
    
    
    # Add any other transformations here...
    
    # Choose a random x,y position for the foreground
    max_xy_position = (background.size[0] - foreground.size[0], background.size[1] - foreground.size[1])
    assert max_xy_position[0] >= 0 and max_xy_position[1] >= 0, \
        'foreground {} is to big for the background {}'.format(foreground_path, background_path)
    paste_position = (random.randint(0, max_xy_position[0]), random.randint(0, max_xy_position[1]))
    
    # Create a new foreground image as large as the background and paste it on top
    new_foreground = Image.new('RGBA', background.size, color = (0, 0, 0, 0))
    new_foreground.paste(foreground, paste_position)
        
    # Extract the alpha channel from the foreground and paste it into a new image the size of the background
    alpha_mask = foreground.getchannel(3)
    new_alpha_mask = Image.new('L', background.size, color=0)
    new_alpha_mask.paste(alpha_mask, paste_position)
    composite = Image.composite(new_foreground, background, new_alpha_mask)
    
    # Grab the alpha pixels above a specified threshold
    alpha_threshold = 200
    mask_arr = np.array(np.greater(np.array(new_alpha_mask), alpha_threshold), dtype=np.uint8)
    hard_mask = Image.fromarray(np.uint8(mask_arr) * 255, 'L')
    
    # Get the smallest & largest non-zero values in each dimension and calculate the bounding box
    nz = np.nonzero(hard_mask)
    bbox = [np.min(nz[0]), np.min(nz[1]), np.max(nz[0]), np.max(nz[1])] 

    return composite, hard_mask, bbox

In [118]:
# Following https://www.immersivelimit.com/tutorials/composing-images-with-python-for-synthetic-datasets
foreground_dir = os.path.join(OUTDIR, 'dom')
background_dir = os.path.join(OUTDIR, 'tables')

output_dir = os.path.join(OUTDIR, 'generated')

foregrounds = glob.glob(foreground_dir+"/*.png")
backgrounds = glob.glob(background_dir+"/*.jpg")

# Generate 5 new images
for i in range(50):
    foreground_path = random.choice(foregrounds)
    background_path = random.choice(backgrounds)
    try:
        composite, mask, bbox = compose_images(foreground_path, background_path)
    except OSError:
        next
        
    
    composite_path = os.path.join(output_dir, 'image_{0:04d}.png'.format(i))
    composite.save(composite_path)
    
    mask_path = os.path.join(output_dir, 'mask_{0:04d}.png'.format(i))
    mask.save(mask_path)



Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling down...
Scaling 

In [119]:
data_iter = mx.image.ImageDetIter(batch_size=4, data_shape=(3, 224, 224),
                                      path_imglist='data/train.lst')

NameError: name 'mx' is not defined

In [64]:
import os
for num, table_url in enumerate(r.text.splitlines()[:10]):
    print(num, table_url, os.path.basename(table_url))

0 http://www.american-furniture-warehouse.com/images/Furniture/stsk4872t.jpg stsk4872t.jpg
1 http://farm3.static.flickr.com/2094/2156394956_636a8e82b7.jpg 2156394956_636a8e82b7.jpg
2 http://farm1.static.flickr.com/119/255091104_0110d3fb49.jpg 255091104_0110d3fb49.jpg
3 http://farm1.static.flickr.com/183/424762396_89fdd67fae.jpg 424762396_89fdd67fae.jpg
4 http://farm4.static.flickr.com/3064/2701347106_5072432f52.jpg 2701347106_5072432f52.jpg
5 http://resaleauctions.com/furniture/diningtable.JPG diningtable.JPG
6 http://farm1.static.flickr.com/19/97833898_b311b6e9cc.jpg 97833898_b311b6e9cc.jpg
7 http://farm4.static.flickr.com/3065/2343394045_bd561e7ab1.jpg 2343394045_bd561e7ab1.jpg
8 http://farm2.static.flickr.com/1093/1423281989_ef0907f396.jpg 1423281989_ef0907f396.jpg
9 http://farm2.static.flickr.com/1085/1481438320_7df49bd11c.jpg 1481438320_7df49bd11c.jpg


In [4]:
import time

query = input("Type the name of the set: ")

time.sleep(0.05)
sets = scrython.sets.Sets()

for i in range(sets.data_length()):
    if sets.set_name(i) == query:
        print("Set code:", sets.set_code(i).upper())
        break
    else:
        continue

Type the name of the set: dom


RuntimeError: Cannot run the event loop while another loop is running